Luke Lawn, Oriana McDonough & Ryan Pike (Group 14)
Reviewing Instructor: Angela Ramnarine-Rieks

Project Overview: Our project will take the top listened artists from a user’s Spotify account and return concerts and venues where that artist is scheduled to perform. This will save people time and allow them to easily see the artists they love listening to performing live, where they can then purchase concert tickets if they choose to. Our goal is to provide users a new way to preview which shows and events they may want to attend based on their personalized listening history.

Inputs: Spotify account username

Outputs: Concert tickets options (price, date, time) and venue information (location, contact information) for top listened artists

Algorithm: 
1 - Import Spotify API and TicketMaster API
2 - User inputs their Spotify account username
3 - Create a list of the user’s top 10 listened artists
4 - For each artist in the list, find upcoming concerts and venue information
5 - Print the above

In [ ]:
import sys
import spotipy
sp = spotipy.Spotify()


In [ ]:
!pip install spotipy

In [ ]:
import sys
import spotipy
import spotipy.util as util

def show_tracks(tracks):
    for i, item in enumerate(tracks['items']):
        track = item['track']
        print(%d %32.32s %s" % (i, track['artists'][0]['name'],track['name'])


if __name__ == '__main__':
    if len(sys.argv) > 1:
        username = sys.argv[1]
    else:
        print "Whoops, need your username!"
        print "usage: python user_playlists.py [username]"
        sys.exit()

    token = util.prompt_for_user_token(username)

    if token:
        sp = spotipy.Spotify(auth=token)
        playlists = sp.user_playlists(username)
        for playlist in playlists['items']:
            if playlist['owner']['id'] == username:
                print
                print playlist['name']
                print '  total tracks', playlist['tracks']['total']
                results = sp.user_playlist(username, playlist['id'],
                    fields="tracks,next")
                tracks = results['tracks']
                show_tracks(tracks)
                while tracks['next']:
                    tracks = sp.next(tracks)
                    show_tracks(tracks)
    else:
        print "Can't get token for", username

In [ ]:
import sys
import spotipy
import spotipy.util as util
username = 'Hockeygoal43@gmail.com'
scope = 'user-library-read'
token = util.prompt_for_user_token(username,scope,client_id='69d411268d5d47c7adc4dc6868928548',client_secret='db95e59f1fa242d28a897684312a85e6',redirect_uri='http://localhost:8888/callback')



if len(sys.argv) > 1:
    username = sys.argv[1]

token = util.prompt_for_user_token(username, scope)

if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    for item in results['items']:
        track = item['track']
        print (track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print("Can't get token for", username)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=69d411268d5d47c7adc4dc6868928548&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Fcallback&scope=user-library-read in your browser


